In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import unidecode

In [3]:
def f(str):
    return (unidecode.unidecode(str))

In [4]:
pd.options.display.float_format = '{:,.2f}'.format

# Remuneração de servidores da Alesp

Fonte: https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2018-05.html

In [17]:
url_chave = ' https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-'
pagamentos = []

In [18]:
for vez in [2015, 2016, 2017, 2018]:
    ano = str(vez)
    for i in range(1, 13):
        if ano == '2018' and i == 6:
            break
        else:
            if i <= 9:
                mes = '0' +  str(i)
            else:
                mes = str(i)
                
            
            link = url_chave + ano + '-' + mes + '.html'
            print(link)
            
            url = urlopen(link)
            
            bsObje = BeautifulSoup(url, "html5lib")
            
            tabela = bsObje.findAll("table", {"class":"tabela_simples"})
            
            conta = 0

            for table_ele in tabela:
                for row in table_ele.findAll('tr'):
                    if conta > 0:
                        cols = row.findAll('td')
                        dicionario = {"ano": ano,
                                      "mes": mes,
                                      "nome_func": cols[0].text.strip(), 
                                      "remuneracao_bruta": cols[1].text.strip(),
                                      "remuneracao_liquida": cols[2].text.strip(),
                                      "tributos": cols[3].text.strip()}
                        pagamentos.append(dicionario)         
                    conta = conta + 1           

 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-01.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-02.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-03.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-04.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-05.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-06.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-07.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-08.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-09.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-10.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-11.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-12.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2016-01.html
 https://www

In [16]:
url = urlopen("https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2018-05.html")
bsObje =  BeautifulSoup(url, "html5lib")

In [5]:
tabela = bsObje.findAll("table", {"class":"tabela_simples"})

In [6]:
pagamentos = []
i = 0
for table_ele in tabela:
    for row in table_ele.findAll('tr'):
        if i > 0:
            cols = row.findAll('td')
            dicionario = {"nome_func": cols[0].text.strip(), 
                              "remuneracao_bruta": cols[1].text.strip(),
                              "remuneracao_liquida": cols[2].text.strip(),
                              "tributos": cols[3].text.strip()}
            pagamentos.append(dicionario)         
        i = i + 1

In [19]:
df_pagamentos = pd.DataFrame(pagamentos)

In [20]:
df_pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201927 entries, 0 to 201926
Data columns (total 6 columns):
ano                    201927 non-null object
mes                    201927 non-null object
nome_func              201927 non-null object
remuneracao_bruta      201927 non-null object
remuneracao_liquida    201927 non-null object
tributos               201927 non-null object
dtypes: object(6)
memory usage: 9.2+ MB


In [21]:
df_pagamentos.reset_index().head()

,index,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos
0,0,2015,01,ABEL PEREIRA,"13.406,63","10.439,20","2.967,43"
1,1,2015,01,ABENOR GARCIA FERREIRA,"17.507,31","13.968,74","3.538,57"
2,2,2015,01,ABILIO DO LIVRAMENTO P ALVES,"11.160,56","7.919,77","3.240,79"
3,3,2015,01,ABNEIA BUFFOLIN BROCANELLI,"4.599,47","3.758,26","841,21"
4,4,2015,01,ABRAO BARBOSA DIB,"9.254,85","7.202,63","2.052,22"


In [22]:
df_pagamentos["nome_func"] = df_pagamentos["nome_func"].apply(f)

In [23]:
df_pagamentos.to_excel('resultados/pagamentos_servidores.xlsx',sheet_name='Sheet1')

In [2]:
df_pagamentos = pd.read_excel('resultados/pagamentos_servidores.xlsx',sheet_name='Sheet1')

In [5]:
df_pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201927 entries, 0 to 201926
Data columns (total 6 columns):
ano                    201927 non-null int64
mes                    201927 non-null int64
nome_func              201927 non-null object
remuneracao_bruta      201927 non-null object
remuneracao_liquida    201927 non-null object
tributos               201927 non-null object
dtypes: int64(2), object(4)
memory usage: 10.8+ MB


# Lotação de servidores

Fonte: https://www.al.sp.gov.br/geral/siga/servidores.jsp?todos=true

In [11]:
url = urlopen("https://www.al.sp.gov.br/geral/siga/servidores.jsp?todos=true")
bsObje =  BeautifulSoup(url, "html5lib")

In [12]:
tabela = bsObje.findAll("table", {"class":"tabela lotacao"})

In [13]:
funcionarios = []
i = 0
for table_ele in tabela:
    for row in table_ele.findAll('tr'):
        if i > 1:
            cols = row.findAll('td')
            dicionario = {"nome_func": cols[0].text.strip(), 
                              "tipo_cargo1": cols[1].text.strip(),
                              "tipo_cargo2": cols[2].text.strip(),
                              "lotacao": cols[3].text.strip()}
            funcionarios.append(dicionario)         
        i = i + 1

In [14]:
df_lotacao = pd.DataFrame(funcionarios)

In [15]:
df_lotacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27854 entries, 0 to 27853
Data columns (total 4 columns):
lotacao        27854 non-null object
nome_func      27854 non-null object
tipo_cargo1    27854 non-null object
tipo_cargo2    27854 non-null object
dtypes: object(4)
memory usage: 870.5+ KB


In [13]:
df_lotacao.reset_index().head()

,index,lotacao,nome_func,tipo_cargo1,tipo_cargo2
0,0,Ex-funcionário,ABDALLAH ABRÃO AUAD,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
1,1,Ex-funcionário,ABDIAS CRISPIM DE ALMEIDA,,
2,2,Ex-funcionário,ABDIAS DA SILVA,AUXILIAR PARLAMENTAR\n\t\t\t\t\t\t\t\n\t\t\t\t...,COMISSÃO
3,3,Ex-funcionário,ABDULNASSER MOHAMAD ABOU NASSIF,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
4,4,Ex-funcionário,ABEDIAS PEREIRA DE ARAUJO,,


In [16]:
df_lotacao["nome_func"] = df_lotacao["nome_func"].apply(f)

In [17]:
df_lotacao.reset_index().head(10)

,index,lotacao,nome_func,tipo_cargo1,tipo_cargo2
0,0,Ex-funcionário,ABDALLAH ABRAO AUAD,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
1,1,Ex-funcionário,ABDIAS CRISPIM DE ALMEIDA,,
2,2,Ex-funcionário,ABDIAS DA SILVA,AUXILIAR PARLAMENTAR\n\t\t\t\t\t\t\t\n\t\t\t\t...,COMISSÃO
3,3,Ex-funcionário,ABDULNASSER MOHAMAD ABOU NASSIF,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
4,4,Ex-funcionário,ABEDIAS PEREIRA DE ARAUJO,,
5,5,,ABEL CASTILHO,COMISSIONADOS,COMISSIONAMENTO SEM PREJUÍZO
6,6,Ex-funcionário,ABEL GARCIA DE SOUZA,,
7,7,Ex-funcionário,ABEL HILARION FERNANDEZ JUNIOR,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
8,8,Ex-funcionário,ABEL PEREIRA,,
9,9,Ex-funcionário,ABEL PUIG PEREIRA,,


In [175]:
df_lotacao.to_excel('resultados/lotacao_servidores.xlsx',sheet_name='Sheet1')

In [6]:
df_lotacao = pd.read_excel('resultados/lotacao_servidores.xlsx',sheet_name='Sheet1')

In [7]:
df_lotacao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27853 entries, 0 to 27852
Data columns (total 4 columns):
lotacao        27842 non-null object
nome_func      27853 non-null object
tipo_cargo1    18510 non-null object
tipo_cargo2    18510 non-null object
dtypes: object(4)
memory usage: 1.1+ MB


In [8]:
df_lotacao['lotacao'].value_counts()

Ex-funcionário                                         24030
LIDERANÇA DO PSDB                                        111
ASSISTÊNCIA POLICIAL MILITAR                              95
1ª SECRETARIA                                             72
SERVIÇO DE CONTROLE DE FROTA                              59
2ª SECRETARIA                                             56
DIVISÃO DE TRANSPORTES                                    52
NAE - NÚCLEO DE AVALIAÇÃO ESTRATÉGICA                     51
LIDERANÇA DO PTB                                          47
LIDERANÇA DO PT                                           44
GABINETE DEP. LUIZ TURCO                                  33
GABINETE DEP. ANA DO CARMO                                32
GABINETE DEP. GILENO GOMES                                32
GABINETE DEP. ENIO TATTO                                  32
GABINETE DEP. TEONILIO BARBA                              32
GABINETE DEP. GERALDO CRUZ                                30
GABINETE DEP. ALENCAR SA

In [9]:
str_choice = "GABINETE" 
gabinetes = df_lotacao[df_lotacao['lotacao'].\
                       str.contains(str_choice, na=False)]

In [10]:
gabinetes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2324 entries, 28 to 27845
Data columns (total 4 columns):
lotacao        2324 non-null object
nome_func      2324 non-null object
tipo_cargo1    2324 non-null object
tipo_cargo2    2324 non-null object
dtypes: object(4)
memory usage: 90.8+ KB


In [11]:
gabinetes['lotacao'].value_counts()

GABINETE DEP. LUIZ TURCO                   33
GABINETE DEP. TEONILIO BARBA               32
GABINETE DEP. ANA DO CARMO                 32
GABINETE DEP. GILENO GOMES                 32
GABINETE DEP. ENIO TATTO                   32
GABINETE DEP. GERALDO CRUZ                 30
GABINETE DEP. PROFESSOR AURIEL             30
GABINETE DEP. ALENCAR SANTANA BRAGA        30
GABINETE DEP. LUIZ FERNANDO T. FERREIRA    30
GABINETE DEP. CORONEL CAMILO               30
GABINETE DEP. CAIO FRANÇA                  29
GABINETE DEP. CARLOS NEDER                 29
GABINETE 48                                29
GABINETE DEP. CÁSSIO NAVARRO               29
GABINETE DEP. ANALICE FERNANDES            29
GABINETE DEP. JORGE CARUSO                 29
GABINETE DEP. GILMACI SANTOS               28
GABINETE DEP. CAUÊ MACRIS                  28
GABINETE DEP. LUIZ CARLOS GONDIM           28
GABINETE DEP. CEZINHA DE MADUREIRA         28
GABINETE DEP. MARCOS DAMASIO               28
GABINETE DEP. ADILSON ROSSI       

In [12]:
gabinetes.reset_index().head(10)

,index,lotacao,nome_func,tipo_cargo1,tipo_cargo2
0,28,GABINETE DEP. CELSO NASCIMENTO,ABNEIA BUFFOLIN BROCANELLI,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
1,30,GABINETE DEP. CELSO NASCIMENTO,ABNER REISES BARBOSA,ASSISTENTE PARLAMENTAR I,COMISSÃO
2,32,GABINETE DEP. RODRIGO MORAES,ABRAAO MOREIRA DA SILVA,ASSISTENTE PARLAMENTAR II,COMISSÃO
3,33,GABINETE DEP. SEBASTIÃO SANTOS,ABRAHAO RICARDO HACKME,ASSISTENTE PARLAMENTAR IV\n\t\t\t\t\t\t\t\n\t\...,COMISSÃO
4,41,GABINETE DEP. CORONEL CAMILO,ACLERISTON PEREIRA MARTINS,ASSISTENTE PARLAMENTAR IV,COMISSÃO
5,69,GABINETE DEP. ANALICE FERNANDES,ADALTO BATISTA DA SILVA,ASSISTENTE PARLAMENTAR II,COMISSÃO
6,99,GABINETE DEP. GUSTAVO PETTA,ADECIR MENDES FONSECA,ASSISTENTE PARLAMENTAR VII,COMISSÃO
7,150,GABINETE DEP. CLÉLIA GOMES,ADEMAR RAYMUNDO GARCIA,ASSISTENTE PARLAMENTAR II,COMISSÃO
8,157,GABINETE DEP. LECI BRANDÃO,ADEMILSON NOGUEIRA DOS SANTOS,ASSISTENTE PARLAMENTAR II,COMISSÃO
9,160,GABINETE DEP. ADILSON ROSSI,ADEMIR ANTONIO DE BARROS,ASSISTENTE TÉCNICO PARLAMENTAR\n\t\t\t\t\t\t\t...,COMISSÃO


In [13]:
len(gabinetes['nome_func'].value_counts())

2321

# Cruza remuneração e lotação

In [14]:
servidores_gabinetes = pd.merge(df_pagamentos, gabinetes, left_on='nome_func', right_on='nome_func')

In [15]:
servidores_gabinetes.reset_index().head()

,index,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos,lotacao,tipo_cargo1,tipo_cargo2
0,0,2015,1,ABNEIA BUFFOLIN BROCANELLI,"4.599,47","3.758,26","841,21",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
1,1,2015,2,ABNEIA BUFFOLIN BROCANELLI,"1.533,16","1.370,40","162,76",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
2,2,2015,3,ABNEIA BUFFOLIN BROCANELLI,"0,00","0,00","0,00",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
3,3,2015,4,ABNEIA BUFFOLIN BROCANELLI,"0,00","-7,43","7,43",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
4,4,2015,5,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.348,13","992,44",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO


Verifica nomes não encontrados na lista de remunerações - apenas 1 deles não é comissionado

In [16]:
nomes_naoencontrados = pd.merge(df_pagamentos,
                        gabinetes,
                        left_on='nome_func', 
                        right_on='nome_func',
                        how='outer',
                        indicator=True)

rdf = nomes_naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)

In [17]:
rdf['nome_func'].value_counts()

EUGENIO CALDEIRA DE OLIVEIRA JABUR              1
MATHEUS VIVEIROS                                1
GABRIEL LEAO MENDES                             1
JULIANA NUNES RIBEIRO CANA                      1
ADOLPHO DOS SANTOS GAMBOA JUNIOR                1
VALDECI GOMES                                   1
LUCIVANIA PORFIRIO DA SILVA                     1
DEBORA DE LIMA SANT'ANNA                        1
ANTONIO BRITO CARDOSO                           1
FELIPE ALENCAR AGUIAR DE CASTRO                 1
RUBENITA DO NASCIMENTO OYAIZU                   1
ISAMARA FARIAS DA SILVA CAMPOS                  1
SILVIA MARIA KURY DE SOUZA                      1
AILTON LINO                                     1
DALMO CARNEIRO FERREIRA                         1
ILMA MARIA CAMPOS FRADE                         1
CEZAR AUGUSTO RAMOS                             1
MAURO ABALEN DE SANT'ANA                        1
DAIANE FARO ZUNO                                1
LUIZ CARLOS DE ALMEIDA PRADO ALVES FILHO        1


In [18]:
rdf['tipo_cargo2'].value_counts()

COMISSÃO                                                                    94
COMISSIONAMENTO SEM PREJUÍZO\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t/ COMISSÃO     1
COMISSIONAMENTO SEM PREJUÍZO                                                 1
EFETIVO                                                                      1
COMISSÃO\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t/ EFETIVO                          1
Name: tipo_cargo2, dtype: int64

In [19]:
nome = rdf[(rdf['tipo_cargo2'] == 'EFETIVO')]
nome

,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos,lotacao,tipo_cargo1,tipo_cargo2
202204,nan,nan,LUIZ CARLOS DE ALMEIDA PRADO ALVES FILHO,NaN,NaN,NaN,GABINETE DEP. FERNANDO CURY,TÉCNICO LEGISLATIVO\n\t\t\t\t\t\t\t\n\t\t\t\t\...,EFETIVO


In [21]:
rdf.to_excel('resultados/servidores_naoencontrados.xlsx',sheet_name='Sheet1')

In [20]:
servidores_gabinetes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62908 entries, 0 to 62907
Data columns (total 9 columns):
ano                    62908 non-null int64
mes                    62908 non-null int64
nome_func              62908 non-null object
remuneracao_bruta      62908 non-null object
remuneracao_liquida    62908 non-null object
tributos               62908 non-null object
lotacao                62908 non-null object
tipo_cargo1            62908 non-null object
tipo_cargo2            62908 non-null object
dtypes: int64(2), object(7)
memory usage: 4.8+ MB


In [39]:
servidores_gabinetes.reset_index().head(10)

,index,nome_func,remuneracao_bruta,remuneracao_liquida,tributos,lotacao,tipo_cargo1,tipo_cargo2
0,0,ABNEIA BUFFOLIN BROCANELLI,"6.519,00","5.125,02","1.393,98",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
1,1,ABNER REISES BARBOSA,"3.092,23","2.643,61","448,62",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR I,COMISSÃO
2,2,ABRAAO MOREIRA DA SILVA,"3.167,73","2.719,98","447,75",GABINETE DEP. RODRIGO MORAES,ASSISTENTE PARLAMENTAR II,COMISSÃO
3,3,ABRAHAO RICARDO HACKME,"9.996,30","7.600,49","2.395,81",GABINETE DEP. SEBASTIÃO SANTOS,ASSISTENTE PARLAMENTAR IV\n\t\t\t\t\t\t\t\n\t\...,COMISSÃO
4,4,ACLERISTON PEREIRA MARTINS,"11.165,63","8.549,85","2.615,78",GABINETE DEP. CORONEL CAMILO,ASSISTENTE PARLAMENTAR IV,COMISSÃO
5,5,ADALTO BATISTA DA SILVA,"3.167,73","2.705,76","461,97",GABINETE DEP. ANALICE FERNANDES,ASSISTENTE PARLAMENTAR II,COMISSÃO
6,6,ADEMAR RAYMUNDO GARCIA,"3.167,73","2.705,76","461,97",GABINETE DEP. CLÉLIA GOMES,ASSISTENTE PARLAMENTAR II,COMISSÃO
7,7,ADEMILSON NOGUEIRA DOS SANTOS,"3.167,73","2.675,10","492,63",GABINETE DEP. LECI BRANDÃO,ASSISTENTE PARLAMENTAR II,COMISSÃO
8,8,ADEMIR ANTONIO DE BARROS,"5.704,06","4.538,08","1.165,98",GABINETE DEP. ADILSON ROSSI,ASSISTENTE TÉCNICO PARLAMENTAR\n\t\t\t\t\t\t\t...,COMISSÃO
9,9,ADEMIR CELESTINO PERETI,"3.167,73","2.705,76","461,97",GABINETE DEP. JOÃO CARAMEZ,ASSISTENTE PARLAMENTAR II,COMISSÃO


In [40]:
def limpeza1(linha):
    valor = linha["remuneracao_bruta"]
    valor = valor.replace('.', '')
    valor = valor.replace(',', '.')
    return valor

def limpeza2(linha):
    valor = linha["remuneracao_liquida"]
    valor = valor.replace('.', '')
    valor = valor.replace(',', '.')
    return valor

In [41]:
servidores_gabinetes["remuneracao_bruta"] = servidores_gabinetes.apply(limpeza1, axis=1)
servidores_gabinetes["remuneracao_liquida"] = servidores_gabinetes.apply(limpeza2, axis=1)

servidores_gabinetes['remuneracao_bruta'] = servidores_gabinetes.remuneracao_bruta.astype('float64')
servidores_gabinetes['remuneracao_liquida'] = servidores_gabinetes.remuneracao_liquida.astype('float64')

In [42]:
servidores_gabinetes.reset_index().head(10)

,index,nome_func,remuneracao_bruta,remuneracao_liquida,tributos,lotacao,tipo_cargo1,tipo_cargo2
0,0,ABNEIA BUFFOLIN BROCANELLI,"6,519.00","5,125.02","1.393,98",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
1,1,ABNER REISES BARBOSA,"3,092.23","2,643.61","448,62",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR I,COMISSÃO
2,2,ABRAAO MOREIRA DA SILVA,"3,167.73","2,719.98","447,75",GABINETE DEP. RODRIGO MORAES,ASSISTENTE PARLAMENTAR II,COMISSÃO
3,3,ABRAHAO RICARDO HACKME,"9,996.30","7,600.49","2.395,81",GABINETE DEP. SEBASTIÃO SANTOS,ASSISTENTE PARLAMENTAR IV\n\t\t\t\t\t\t\t\n\t\...,COMISSÃO
4,4,ACLERISTON PEREIRA MARTINS,"11,165.63","8,549.85","2.615,78",GABINETE DEP. CORONEL CAMILO,ASSISTENTE PARLAMENTAR IV,COMISSÃO
5,5,ADALTO BATISTA DA SILVA,"3,167.73","2,705.76","461,97",GABINETE DEP. ANALICE FERNANDES,ASSISTENTE PARLAMENTAR II,COMISSÃO
6,6,ADEMAR RAYMUNDO GARCIA,"3,167.73","2,705.76","461,97",GABINETE DEP. CLÉLIA GOMES,ASSISTENTE PARLAMENTAR II,COMISSÃO
7,7,ADEMILSON NOGUEIRA DOS SANTOS,"3,167.73","2,675.10","492,63",GABINETE DEP. LECI BRANDÃO,ASSISTENTE PARLAMENTAR II,COMISSÃO
8,8,ADEMIR ANTONIO DE BARROS,"5,704.06","4,538.08","1.165,98",GABINETE DEP. ADILSON ROSSI,ASSISTENTE TÉCNICO PARLAMENTAR\n\t\t\t\t\t\t\t...,COMISSÃO
9,9,ADEMIR CELESTINO PERETI,"3,167.73","2,705.76","461,97",GABINETE DEP. JOÃO CARAMEZ,ASSISTENTE PARLAMENTAR II,COMISSÃO


Gabinetes com gastos

In [43]:
a = len(servidores_gabinetes['lotacao'].value_counts())
a

102

Soma dos gastos com servidores

In [27]:
"${:,.2f}".format(servidores_gabinetes.remuneracao_bruta.sum())

'$18,251,334.94'

Média

In [28]:
servidores_gabinetes['remuneracao_bruta'].mean()

8184.455130044844

Mediana

In [30]:
"${:,.0f}".format(servidores_gabinetes.remuneracao_bruta.median())

'$6,335'

In [45]:
servidores_gabinetes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2230 entries, 0 to 2229
Data columns (total 7 columns):
nome_func              2230 non-null object
remuneracao_bruta      2230 non-null float64
remuneracao_liquida    2230 non-null float64
tributos               2230 non-null object
lotacao                2230 non-null object
tipo_cargo1            2230 non-null object
tipo_cargo2            2230 non-null object
dtypes: float64(2), object(5)
memory usage: 139.4+ KB


Gastos com servidores por gabinete - total mensal

In [51]:
servidores_gabinetes.groupby("lotacao").remuneracao_bruta.sum().reset_index().sort_values("remuneracao_bruta", ascending=False)

,lotacao,remuneracao_bruta
9,GABINETE DEP. ALDO DEMARCHI,"286,741.94"
11,GABINETE DEP. ANA DO CARMO,"260,678.69"
44,GABINETE DEP. FERNANDO CAPEZ,"258,273.27"
39,GABINETE DEP. EDMIR CHEDID,"250,640.69"
61,GABINETE DEP. LUIZ CARLOS GONDIM,"246,786.77"
63,GABINETE DEP. LUIZ TURCO,"246,324.01"
78,GABINETE DEP. PEDRO TOBIAS,"241,900.06"
91,GABINETE DEP. ROGÉRIO NOGUEIRA,"237,100.94"
90,GABINETE DEP. RODRIGO MORAES,"230,465.92"
80,GABINETE DEP. RAFAEL SILVA,"226,501.73"


Porcentagens em relação ao total - Gastos com servidores por gabinete - total mensal

In [40]:
# Agrupamento dos gastos por categoria
df_gabinete = servidores_gabinetes.groupby(['lotacao']).agg({'remuneracao_bruta': sum})\
                                .sort_values('remuneracao_bruta', ascending = False)\
                                .reset_index()

# Cálculo de porcentagem de cada categoria no total de gastos
df_gabinete['porcentagem'] = (df_gabinete['remuneracao_bruta'] / df_gabinete['remuneracao_bruta'].sum()) * 100

# Formatação de números para duas casas decimais
df_gabinete['remuneracao_bruta']['remuneracao_bruta'] = df_gabinete['remuneracao_bruta'].map('{:,.2f}'.format)
df_gabinete['porcentagem'] = df_gabinete['porcentagem'].map('{:,.2f}'.format)

# Resultado
df_gabinete.reset_index().head(10)

/home/reinaldo/Documentos/Code/volt/alesp/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,index,lotacao,remuneracao_bruta,porcentagem
0,0,GABINETE DEP. ALDO DEMARCHI,"286,741.94",1.57
1,1,GABINETE DEP. ANA DO CARMO,"260,678.69",1.43
2,2,GABINETE DEP. FERNANDO CAPEZ,"258,273.27",1.42
3,3,GABINETE DEP. EDMIR CHEDID,"250,640.69",1.37
4,4,GABINETE DEP. LUIZ CARLOS GONDIM,"246,786.77",1.35
5,5,GABINETE DEP. LUIZ TURCO,"246,324.01",1.35
6,6,GABINETE DEP. PEDRO TOBIAS,"241,900.06",1.33
7,7,GABINETE DEP. ROGÉRIO NOGUEIRA,"237,100.94",1.30
8,8,GABINETE DEP. RODRIGO MORAES,"230,465.92",1.26
9,9,GABINETE DEP. RAFAEL SILVA,"226,501.73",1.24


Quantidade de servidores por gabinete

In [46]:
servidores_gabinetes.groupby("lotacao").nome_func.count().sort_values(ascending=False)

lotacao
GABINETE DEP. ANA DO CARMO                 34
GABINETE DEP. LUIZ TURCO                   33
GABINETE DEP. GILENO GOMES                 32
GABINETE DEP. ALENCAR SANTANA BRAGA        32
GABINETE DEP. ENIO TATTO                   32
GABINETE DEP. TEONILIO BARBA               31
GABINETE DEP. CAIO FRANÇA                  30
GABINETE DEP. PROFESSOR AURIEL             30
GABINETE DEP. CORONEL CAMILO               30
GABINETE DEP. LUIZ CARLOS GONDIM           29
GABINETE DEP. ANALICE FERNANDES            29
GABINETE DEP. GERALDO CRUZ                 29
GABINETE 48                                29
GABINETE DEP. LUIZ FERNANDO T. FERREIRA    28
GABINETE DEP. JOSÉ ZICO PRADO              28
GABINETE DEP. CEZINHA DE MADUREIRA         28
GABINETE DEP. CARLOS NEDER                 28
GABINETE DEP. MARCOS DAMASIO               28
GABINETE DEP. JORGE CARUSO                 28
GABINETE DEP. ADILSON ROSSI                27
GABINETE DEP. CAUÊ MACRIS                  27
GABINETE DEP. RODRIGO MORA

In [47]:
servidores_gabinetes.groupby("nome_func").remuneracao_bruta.sum().reset_index().sort_values("remuneracao_bruta", ascending=False).head(10)

,nome_func,remuneracao_bruta
997,JOAO CARLOS FRANCISCO DA SILVA,"66,018.76"
118,ANA MARIA DA SILVA,"48,959.16"
1829,RICARDO DE OLIVEIRA,"29,920.90"
959,JANETE DA CRUZ LION,"25,322.25"
1078,JOSE CAVALLI JUNIOR,"25,322.25"
416,CLAUDIO FERREIRA,"25,322.25"
1499,MARIA TERESA DOS SANTOS SIQUEIRA,"25,322.25"
1250,LUCIA HELENA JUVENCIO,"25,322.25"
687,EVALDO PEREIRA DA SILVA,"25,322.25"
150,ANDRE LUIS CHUFULI AMOROSO,"25,322.25"
